In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

# openai
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# huggingface
HF_TOKEN = os.getenv("HF_ACCESS_TOKEN")

# google
GOOGLE_CSE_ID = os.getenv("GOOGLE_CSE_ID")
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
SERPAPI_KEY = os.getenv("SERPAPI_API_KEY")

# Things to try

In [ ]:
https://python.langchain.com/docs/modules/data_connection/retrievers/web_research
https://www.mlq.ai/gpt-3-enabled-research-assistant-langchain-pinecone/
https://python.langchain.com/docs/integrations/tools/wikipedia

# German laws to english summary

HTML2Text converts HTML into plain text (with markdown-like formatting) without any specific tag manipulation.
To extract human-readable text without needing to manipulate specific HTML elements.

In [ ]:
import openai
from langchain.document_loaders import AsyncHtmlLoader
from langchain.document_transformers import Html2TextTransformer
from translate import Translator

urls = ["https://www.gesetze-im-internet.de/baf_gzuschlagsv/BJNR009350986.html",
        "https://www.gesetze-im-internet.de/baf_g/BJNR014090971.html"
        ]

loader = AsyncHtmlLoader(urls)
docs = loader.load()

html2text = Html2TextTransformer()
docs_transformed = html2text.transform_documents(docs)


# Language Translation
translator = Translator(to_lang='en', from_lang='de')
german_text = docs_transformed[0].page_content  # Assuming there's one document
english_text = translator.translate(german_text[:500])

# summarization
def summarize_text(text, person_type):
    res = openai.ChatCompletion.create(
        model="gpt-3.5-turbo", max_tokens=1000, temperature=0.7, top_p=0.5, frequency_penalty=0.5,
        messages=
       [ { "role": "system", "content": "You are a helpful assistant for text summarization.", },
         { "role": "user", "content": f"Summarize this for a {person_type}: {text}", }, ],
    )
    return res["choices"][0]["message"]["content"]

english_summary = summarize_text(english_text, 'person without legal training')
english_summary

# Identify Paragraph References
# You might need a parser specific to German legal texts to extract paragraph references.

# Include References in the Summary
# Incorporate the references into the English summary.

# Output
print(english_summary)

# Web retriever

In [6]:
import logging

from langchain.chains import RetrievalQAWithSourcesChain
from langchain.chat_models.openai import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.retrievers.web_research import WebResearchRetriever
from langchain.utilities import GoogleSearchAPIWrapper
from langchain.vectorstores import Chroma
import nest_asyncio
nest_asyncio.apply()

# Vectorstore
vectorstore = Chroma(
    embedding_function=OpenAIEmbeddings(),
    persist_directory="./chroma_db_oai",
)

# LLM
llm = ChatOpenAI( temperature=0 )

# Custom Search with Google Programmable Search Engine
search = GoogleSearchAPIWrapper()

# Initialize
web_research_retriever = WebResearchRetriever.from_llm(
    vectorstore=vectorstore,
    llm=llm,
    search=search,
)

# logging
logging.basicConfig()
logging.getLogger("langchain.retrievers.web_research").setLevel(logging.INFO)

# retrieve docs and provide citations
qa_chain = RetrievalQAWithSourcesChain.from_chain_type(
    llm, retriever=web_research_retriever
)

user_input = "Where can I find laws regulating the duties of landlords in Berlin Germany?"
docs = web_research_retriever.get_relevant_documents(user_input)

result = qa_chain({"question": user_input})
result

## WebRetriever with custom prompt and output parsing.

In [19]:
import os
import re
from typing import List
from langchain.chains import LLMChain
from pydantic import BaseModel, Field
from langchain.prompts import PromptTemplate
from langchain.output_parsers.pydantic import PydanticOutputParser

# LLMChain
search_prompt = PromptTemplate(
    input_variables=["question"],
    template="""You are an assistant tasked with improving Google search 
    results. Generate FIVE Google search queries that are similar to
    this question. Transalte your queries into german as your searching for german official documents.
    Return the sources of your search.
    The output should be a numbered list of questions and each
    should have a question mark at the end: {question}""",
)

class LineList(BaseModel):
    """List of questions."""

    lines: List[str] = Field(description="Questions")

class QuestionListOutputParser(PydanticOutputParser):
    """Output parser for a list of numbered questions."""

    def __init__(self) -> None:
        super().__init__(pydantic_object=LineList)

    def parse(self, text: str) -> LineList:
        lines = re.findall(r"\d+\..*?\n", text)
        return LineList(lines=lines)
    
llm_chain = LLMChain(
            llm=llm,
            prompt=search_prompt,
            output_parser=QuestionListOutputParser(),
        )

search = GoogleSearchAPIWrapper()

# Initialize
web_research_retriever_llm_chain = WebResearchRetriever(
    vectorstore=vectorstore,
    llm_chain=llm_chain, 
    search=search, 
)


# logging
logging.basicConfig()
logging.getLogger("langchain.retrievers.web_research").setLevel(logging.INFO)

# retrieve docs and provide citations
qa_chain = RetrievalQAWithSourcesChain.from_chain_type(
    llm, retriever=web_research_retriever, 
)


# Run
user_input = "Where can I find laws regulating the duties of landlords in Berlin Germany?"
docs = web_research_retriever_llm_chain.get_relevant_documents(user_input)
result = qa_chain({"question": user_input})
result

Fetching pages: 100%|##########| 1/1 [00:00<00:00,  1.72it/s]
INFO:langchain.retrievers.web_research:Generating questions for Google Search ...
INFO:langchain.retrievers.web_research:Questions for Google Search (raw): {'question': 'Where can I find laws regulating the duties of landlords in Berlin Germany?', 'text': LineList(lines=['1. What are the legal obligations of landlords in Berlin, Germany?\n', '2. Where can I access information on landlord responsibilities in Berlin, Germany?\n', '3. What are the laws governing landlord duties in Berlin, Germany?'])}
INFO:langchain.retrievers.web_research:Questions for Google Search: ['1. What are the legal obligations of landlords in Berlin, Germany?\n', '2. Where can I access information on landlord responsibilities in Berlin, Germany?\n', '3. What are the laws governing landlord duties in Berlin, Germany?']
INFO:langchain.retrievers.web_research:Searching for relevant urls...
INFO:langchain.retrievers.web_research:Searching for relevant url

{'question': 'Where can I find laws regulating the duties of landlords in Berlin Germany?',
 'answer': 'You can find laws regulating the duties of landlords in Berlin, Germany in the following ',
 'sources': ''}

# Google Programmable Search 
https://cse.google.com/cse.js?cx=8566b4b2cb2364df3

In [21]:
from langchain.tools import Tool
from langchain.utilities import GoogleSearchAPIWrapper



def search(query):
    search = GoogleSearchAPIWrapper(google_api_key=GOOGLE_API_KEY, google_cse_id=GOOGLE_CSE_ID)
    return search.results(query, 10)


tool = Tool(
    name="Google Search Snippets",
    description="Search Google for recent results.",
    func=search,
)


tool.run("Wohnungsaufsichtsgesetz?")

[{'title': 'Wohnungsaufsichtsgesetz – Wikipedia',
  'link': 'https://de.wikipedia.org/wiki/Wohnungsaufsichtsgesetz',
  'snippet': 'Ein Wohnungsaufsichtsgesetz regelt Pflichten der Vermieter. Deutschland Bearbeiten. In Deutschland existieren in einigen Bundesländern\xa0...'},
 {'title': 'HSGV Inhalt : Historisch: Wohnungsaufsichtsgesetz (WAG NRW ...',
  'link': 'https://recht.nrw.de/lmi/owa/br_bes_text?bes_id=26926&aufgehoben=J&anw_nr=2',
  'snippet': '(1) Wohnraum darf nur überlassen oder benutzt werden, wenn für jede Bewohnerin oder jeden Bewohner eine Wohnfläche von mindestens 9 m², für jedes Kind bis sechs\xa0...'},
 {'title': 'Wohnungsaufsicht - Berlin.de',
  'link': 'https://www.berlin.de/sen/bauen/baurecht-und-bauplanung/bauaufsicht/wohnungsaufsicht/',
  'snippet': 'Wohnraum soll lebenswert sein und den Bewohnerinnen und Bewohnern ein sicheres zu Hause bieten. Ziel des Wohnungsaufsichtsrechts ist es, für die Beseitigung\xa0...'},
 {'title': 'Mindestwohnfläche pro Person in Mietwo

# Plan-and-execute

In [72]:
from langchain.chat_models import ChatOpenAI
from langchain_experimental.plan_and_execute import (
    PlanAndExecute,
    load_agent_executor,
    load_chat_planner,
)
from langchain.llms import OpenAI
from langchain import SerpAPIWrapper
from langchain.agents.tools import Tool
from langchain import LLMMathChain

In [75]:
search = SerpAPIWrapper()

llm = OpenAI(temperature=0)

llm_math_chain = LLMMathChain.from_llm(llm=llm, verbose=True)

tools = [
    Tool(
        name="Search",
        func=search.run,
        description="Find offical german documents. translate the user input into german and search for pdf, doc, docx files and provide their sources?",
    ),
]

model = ChatOpenAI(temperature=0)
planner = load_chat_planner(model)
executor = load_agent_executor(model, tools, verbose=True)
agent = PlanAndExecute(planner=planner, executor=executor, verbose=True)

agent.run(
    "What law regulates the duties of a landlord in berlin?"
)



> Entering new PlanAndExecute chain...
steps=[Step(value='Research the legal system in Berlin to understand how landlord-tenant relationships are regulated.'), Step(value='Identify the specific law or laws that govern the duties of landlords in Berlin.'), Step(value='Review the relevant law(s) to understand the specific duties and responsibilities imposed on landlords.'), Step(value='Summarize the key provisions of the law(s) that regulate the duties of landlords in Berlin.'), Step(value='Given the above steps taken, provide the user with the information about the specific law that regulates the duties of a landlord in Berlin.')]

> Entering new AgentExecutor chain...
Action:
```
{
  "action": "Search",
  "action_input": "Legal system in Berlin landlord-tenant regulations"
}
```
Observation: Rental laws in Germany largely favor tenants over landlords and offer extensive protection under a very robust legal framework. However, there are various ...
Thought:I have found some informatio

'I have searched for official German documents that specifically address the law or laws that regulate the duties of landlords in Berlin. Here is some information I found:\n\n- German Tenancy Law (Mietrecht) is the main legislation that governs landlord-tenant relationships in Germany, including Berlin. It is regulated under the German Civil Code (Bürgerliches Gesetzbuch or BGB).\n\n- The specific provisions regarding the duties of landlords can be found in Sections 535 to 580a of the BGB. These provisions outline the obligations of landlords, such as maintaining the rental property, addressing defects or damages, providing a habitable living space, and ensuring the privacy of the tenant.\n\n- Additionally, the Berlin Tenancy Act (Berliner Mietspiegel) provides further regulations on rent levels and rent increases in Berlin.\n\nFor more detailed and accurate information, I recommend consulting the official German Tenancy Law (Mietrecht) and the German Civil Code (BGB). These documents 

# Tools

## PSE & Wiki 

In [24]:
from langchain import OpenAI
from langchain.agents import AgentType, Tool, initialize_agent, load_tools
from langchain.llms.openai import OpenAI
from langchain.tools import Tool, WikipediaQueryRun
from langchain.utilities import (
    GoogleSearchAPIWrapper,
    WikipediaAPIWrapper,

)


def pse(query):
    search = GoogleSearchAPIWrapper(google_api_key=GOOGLE_API_KEY, google_cse_id=GOOGLE_CSE_ID)
    return search.results(query, 5)

wiki = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())


tools = [
    Tool(
        name="pse",
        description="use this tool first. SEARCH IN GERMAN. Find offical german documents and laws concerning the users problem.\
            Retrieve sources and links to the document and give a short summary of its content.\
            Find official documents of Germany, preferably in pdf fromat",
        func=pse,
    ),

    Tool(
        name="wiki",
        description="use this tool when the first failed. \
            Find official documents of Germany, preferably in pdf fromat\
            Look in the footnotes and links of wikipedia pages for links to pdf's and sources\
            follow the links and find the source where the file can be downloaded",
        func=wiki.run,
    ),
]

agent = initialize_agent(
    tools,
    OpenAI(temperature=0),
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)


agent.run(
    "What law regulates the duties of a landlord in berlin?"
)



> Entering new AgentExecutor chain...
 I should look for official documents related to this topic
Action: pse
Action Input: "landlord duties berlin"
Observation: [{'title': 'Landlord and tenant obligations in Germany - Berlin', 'link': 'https://aden-immo.com/en/landlord-guide/landlord-and-tenant-obligations-in-germany/', 'snippet': 'Discover the rights and duties of tenants and their landlord ✓✓✓ Find out everything there is to know before renting your property!'}, {'title': 'Rent price cap: you should observe these obligations as a landlord ...', 'link': 'https://www.engelvoelkers.com/en-de/berlincommercial/rent-price-cap-obligations-for-owners-landlords/', 'snippet': 'UPDATE: The Berlin rent cap was declared invalid by a decision of the Federal Constitutional Court in Karlsruhe on April 15, 2021.'}, {'title': "What you need to know about German renters' rights", 'link': 'https://www.lemonade.com/de/blog/renters-rights/', 'snippet': 'Not only landlords have to fulfill their obligati

'The law that regulates the duties of a landlord in Berlin is the German rental law, which is outlined in the rental contract (Mietvertrag) that must be signed by both the landlord and the tenant. The contract must contain the rights and obligations of both parties, as well as the terms and conditions of the rental agreement.'

# Serper, serapi, google-search, searchNG

In [79]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.llms import OpenAI

llm = OpenAI(temperature=0)


tools = load_tools(["google-serper", "serpapi", "google-search", "searx-search"], searx_host="http://localhost:8888", llm=llm)

agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

agent.run("Where can I find the law regulating the duties of a landlord in berlin germany?")



> Entering new AgentExecutor chain...
 I should search for the law online
Action: google_search
Action Input: "landlord duties berlin germany"
Observation: Full text in format: HTML PDF · text in German ; Section 239Surety ; Section 240Duty to supplement security ; Book 2. Law of obligations ; Division 1. Subject ... Legal obligations of secrecy, especially tax secrecy pursuant to Section 30 of the German Fiscal Code, as well as professional or special official secrecy ... 1 Act of 30.6.1959 101-3; Entry into force in Berlin on 10.8.1951 pursuant ... Where these duties are performed by an expert, the provisions of the General ...
Thought: I should look for a specific law
Action: google_search
Action Input: "landlord duties berlin germany law"
Observation: Full text in format: HTML PDF · text in German ; Section 239Surety ; Section 240Duty to supplement security ; Book 2. Law of obligations ; Division 1. Subject ... Book 10 of the German Code of Civil Procedure is largely based on the

'The law regulating the duties of a landlord in Berlin, Germany is found in Book 2 of the German Code of Civil Procedure, Section 239Surety, Section 240Duty to supplement security, and Book 10 of the German Code of Civil Procedure. Additionally, the German Civil Code and the Act of 30.6.1959 101-3 are also relevant.'

# Langchain + Streamlit Searchbar

In [ ]:
import streamlit as st

from langchain.agents import initialize_agent, AgentType
from langchain.callbacks import StreamlitCallbackHandler
from langchain.chat_models import ChatOpenAI
from langchain.tools import DuckDuckGoSearchRun

with st.sidebar:
    openai_api_key = st.text_input(
        "OpenAI API Key", key="langchain_search_api_key_openai", type="password"
    )
    "[Get an OpenAI API key](https://platform.openai.com/account/api-keys)"
    "[View the source code](https://github.com/streamlit/llm-examples/blob/main/pages/2_Chat_with_search.py)"
    "[![Open in GitHub Codespaces](https://github.com/codespaces/badge.svg)](https://codespaces.new/streamlit/llm-examples?quickstart=1)"

st.title("🔎 LangChain - Chat with search")

"""
In this example, we're using `StreamlitCallbackHandler` to display the thoughts and actions of an agent in an interactive Streamlit app.
Try more LangChain 🤝 Streamlit Agent examples at [github.com/langchain-ai/streamlit-agent](https://github.com/langchain-ai/streamlit-agent).
"""

if "messages" not in st.session_state:
    st.session_state["messages"] = [
        {
            "role": "assistant",
            "content": "Hi, I'm a chatbot who can search the web. How can I help you?",
        }
    ]

for msg in st.session_state.messages:
    st.chat_message(msg["role"]).write(msg["content"])

if prompt := st.chat_input(placeholder="Who won the Women's U.S. Open in 2018?"):
    st.session_state.messages.append({"role": "user", "content": prompt})
    st.chat_message("user").write(prompt)

    if not openai_api_key:
        st.info("Please add your OpenAI API key to continue.")
        st.stop()

    llm = ChatOpenAI(
        model_name="gpt-3.5-turbo", openai_api_key=openai_api_key, streaming=True
    )
    search = DuckDuckGoSearchRun(name="Search")
    search_agent = initialize_agent(
        [search],
        llm,
        agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
        handle_parsing_errors=True,
    )
    with st.chat_message("assistant"):
        st_cb = StreamlitCallbackHandler(st.container(), expand_new_thoughts=False)
        response = search_agent.run(st.session_state.messages, callbacks=[st_cb])
        st.session_state.messages.append({"role": "assistant", "content": response})
        st.write(response)